In [1]:
from agents import Agent, WebSearchTool, trace, Runner, gen_trace_id, function_tool
from agents.model_settings import ModelSettings
from pydantic import BaseModel, Field
from dotenv import load_dotenv
import asyncio
import sendgrid
import os
from sendgrid.helpers.mail import Mail, Email, To, Content
from typing import Dict
from IPython.display import display, Markdown


In [2]:
load_dotenv(override=True)


True

In [3]:
INSTRUCTIONS = """You are a tech news researcher specializing in finding viral and interesting technology stories. 
Given a search term, you search the web and produce a concise summary that highlights what makes the story interesting 
and potentially viral. Focus on:
1. Why this story is capturing attention
2. Key surprising or controversial elements
3. Social media reaction and engagement
4. Potential future implications

The summary must be 2-3 paragraphs and less than 300 words. Write in an engaging style that draws readers in 
while maintaining accuracy. Include relevant metrics like view counts, shares, or trending status if available.

IMPORTANT: You MUST include the source URL for each story in markdown link format at the end of your summary, like this:
[Source: Publication Name](https://url-to-story)

If there are multiple key sources, include them all in separate markdown links. Never mention a story without its source link."""

search_agent = Agent(
    name="Search agent",
    instructions=INSTRUCTIONS,
    tools=[WebSearchTool(search_context_size="low")],
    model="gpt-4o-mini",
    model_settings=ModelSettings(tool_choice="required"),
)


In [4]:
HOW_MANY_SEARCHES = 5  # Increased from 3 to get more diverse news coverage

INSTRUCTIONS = f"""You are a viral tech news curator. Given a request for interesting tech news, 
come up with {HOW_MANY_SEARCHES} specific search queries that will help find the most engaging and 
viral tech stories of the week. Focus on:
1. Breaking tech news
2. Controversial tech developments
3. Viral tech demos or releases
4. Surprising announcements or discoveries
5. Tech stories gaining significant social media traction"""

class WebSearchItem(BaseModel):
    reason: str = Field(description="Why this search will help find viral/interesting tech news")
    query: str = Field(description="The search term to use for finding viral tech news")

class WebSearchPlan(BaseModel):
    searches: list[WebSearchItem] = Field(description="A list of searches to find viral tech news")

planner_agent = Agent(
    name="PlannerAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=WebSearchPlan,
)


In [5]:
@function_tool
def send_email(subject: str, html_body: str) -> Dict[str, str]:
    """ Send out an email with the given subject and HTML body """
    sg = sendgrid.SendGridAPIClient(api_key=os.environ.get('SENDGRID_API_KEY'))
    from_email = Email("sanders.brett@gmail.com")  # Change this to your verified email
    to_email = To("blogbrett@gmail.com")  # Change this to your email
    content = Content("text/html", html_body)
    mail = Mail(from_email, to_email, subject, content).get()
    response = sg.client.mail.send.post(request_body=mail)
    return {"status": "success"}


In [6]:
INSTRUCTIONS = """You are a tech news email curator. You will be provided with a detailed report about viral 
tech news. Convert this into an engaging HTML email that will make readers want to click and read more.
Use attention-grabbing subject lines, clear formatting, and highlight the most interesting aspects of each story.

CRITICAL: Source links are essential to this email's value:
- Convert ALL markdown links to proper HTML <a> tags with appropriate styling
- Every story MUST maintain its source attribution link(s)
- Style links to be clearly visible and inviting to click
- When multiple sources are cited for a story, preserve ALL links
- Double-check that no links are lost in the markdown-to-HTML conversion

The email should be visually appealing with clear sections, highlights, and proper spacing."""

email_agent = Agent(
    name="Email agent",
    instructions=INSTRUCTIONS,
    tools=[send_email],
    model="gpt-4o-mini",
)


In [7]:
INSTRUCTIONS = """You are a senior tech journalist tasked with writing an engaging report about the most 
interesting tech news of the week. You will be provided with research about various viral tech stories.

Create a report that:
1. Ranks stories by their viral potential and significance
2. Analyzes why each story is capturing attention
3. Provides context and implications
4. Includes relevant social media reactions and metrics
5. Makes predictions about how these stories might evolve

CRITICAL REQUIREMENTS:
- The report should be in markdown format, engaging but accurate, and aim for 1000-1500 words
- EVERY story mentioned MUST include its source link(s) in markdown format: [Source: Publication Name](url)
- When aggregating information from multiple sources about the same story, include ALL source links
- Place source links immediately after discussing each story
- Double-check that all URLs from the research are preserved and properly formatted
- Never discuss a story without providing its source link"""

class ReportData(BaseModel):
    short_summary: str = Field(description="A catchy 2-3 sentence summary of the top viral tech stories")
    markdown_report: str = Field(description="The detailed analysis of viral tech news")
    follow_up_questions: list[str] = Field(description="Emerging angles or developments to watch")

writer_agent = Agent(
    name="WriterAgent",
    instructions=INSTRUCTIONS,
    model="gpt-4o-mini",
    output_type=ReportData,
)


In [8]:
async def plan_searches(query: str):
    """ Use the planner_agent to plan which searches to run for viral tech news """
    print("Planning searches for viral tech news...")
    result = await Runner.run(planner_agent, f"Query: {query}")
    print(f"Will perform {len(result.final_output.searches)} searches")
    return result.final_output

async def perform_searches(search_plan: WebSearchPlan):
    """ Call search() for each item in the search plan """
    print("Searching for viral tech stories...")
    tasks = [asyncio.create_task(search(item)) for item in search_plan.searches]
    results = await asyncio.gather(*tasks)
    print("Finished searching")
    return results

async def search(item: WebSearchItem):
    """ Use the search agent to find viral tech news """
    input = f"Search term: {item.query}\nLooking for: {item.reason}"
    result = await Runner.run(search_agent, input)
    return result.final_output


In [9]:
async def write_report(query: str, search_results: list[str]):
    """ Use the writer agent to create an engaging report about viral tech news """
    print("Analyzing viral tech stories...")
    input = f"Original query: {query}\nResearch findings: {search_results}"
    result = await Runner.run(writer_agent, input)
    print("Finished writing viral tech report")
    return result.final_output

async def send_email(report: ReportData):
    """ Use the email agent to send an engaging email about viral tech news """
    print("Crafting viral tech news email...")
    result = await Runner.run(email_agent, report.markdown_report)
    print("Email sent")
    return report


In [10]:
query = "Most viral and interesting technology news this week"

with trace("Viral Tech News Research"):
    print("Starting viral tech news research...")
    search_plan = await plan_searches(query)
    search_results = await perform_searches(search_plan)
    report = await write_report(query, search_results)
    await send_email(report)  
    print("Viral tech news report complete and sent!")


Starting viral tech news research...
Planning searches for viral tech news...
Will perform 5 searches
Searching for viral tech stories...
Finished searching
Analyzing viral tech stories...
Finished writing viral tech report
Crafting viral tech news email...
Email sent
Viral tech news report complete and sent!
